In [ ]:
%load configuration.py


In [ ]:
%load data.py


In [ ]:
%load setup-pd.py


In [ ]:
# Build key set
# ================
#
# Build the set of keys such that it contains every network type and each combination of it.
keys = [x for x in origins]

for i in range(len(origins)):
    for j in range(i + 1, len(origins)):
        keys.append(f"{origins[i]} {origins[j]}")

keys.append(" ".join(origins))

keys

In [ ]:
# IPs of different networks
# ================

# Get the distinct IP addresses of the different networks

df = df_log[df_log.category == 'ssh']
df = df[['origin', 'origin_id', 'timestamp', 'ip']]
df = df.merge(df_ip, how='left', left_on='ip', right_on='ip')
df = df[['origin', 'origin_id', 'ip', 'longitude', 'latitude', 'timestamp']]
df = df.groupby(['origin', 'origin_id', 'ip', 'longitude', 'latitude']).count().rename(columns={'timestamp': 'requests'})
df = df.reset_index()
df


In [ ]:
plots = {}

# Plots for each network

for k in origins:
    tmp = df[df.origin == k][['ip', 'longitude', 'latitude', 'requests']]
    tmp = tmp.set_index('ip')
    plots[k] = tmp

# Plot for the intersections of networks

import sqlite3
con = sqlite3.connect(":memory:")

for k in plots.keys():
    plots[k].to_sql(k, con, index=True)


query = "SELECT ip, longitude, latitude FROM cloud INTERSECT SELECT ip, longitude, latitude FROM campus;"
plots['cloud campus'] = pd.read_sql_query(query, con)

query = "SELECT ip, longitude, latitude FROM cloud INTERSECT SELECT ip, longitude, latitude FROM residential;"
plots['cloud residential'] = pd.read_sql_query(query, con)

query = "SELECT ip, longitude, latitude FROM campus INTERSECT SELECT ip, longitude, latitude FROM residential;"
plots['campus residential'] = pd.read_sql_query(query, con)

query = "SELECT ip, longitude, latitude FROM cloud INTERSECT SELECT ip, longitude, latitude FROM campus INTERSECT SELECT ip, longitude, latitude FROM residential;"
plots['cloud campus residential'] = pd.read_sql_query(query, con)


# Calculate all requests per plot.

summary = df.groupby(['origin', 'ip', 'longitude', 'latitude']).sum()
summary = summary.reset_index()[['ip', 'origin', 'longitude', 'latitude', 'requests']]

for k in plots.keys():
    orgs = k.split(' ')
    #plots[k]['requests'] = summary[summary.origin.isin(orgs)].groupby(['longitude', 'latitude'], dropna=False).sum()
    tmp = summary[summary.origin.isin(orgs)].groupby(['ip', 'longitude', 'latitude'], dropna=False).sum()
    tmp = tmp.reset_index()
    
    ips = plots[k].reset_index().ip.unique()
    
    tmp = tmp[tmp.ip.isin(ips)].groupby(['ip', 'longitude', 'latitude']).sum()
    tmp = tmp.reset_index().set_index('ip')
    plots[k] = tmp

#pd.concat(plots.values())
#summary = summary.groupby(['ip', 'longitude', 'latitude', 'requests'], dropna=False).sum()
#summary

#or k in plots.values():
    

#for k in plots.keys():
#    tmp = plots[k]
#    tmp['requests'] = 

In [ ]:
for k, v in plots.items():
    # Should we limit those scanners with less than X login attempts?
    #vmin, vmean, vmax = v.requests.min(), v.requests.mean(), v.requests.max()
    #v = v[v.requests > 10]
    plots[k] = v.sort_values(by=['requests'], ascending=True)

In [ ]:
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))

for k, v in plots.items():
    fig, ax = plt.subplots(
        figsize=(8,6),
    )
    
    world.plot(
        color='darkgrey',
        ax=ax,
    )
    
    parts = k.split(" ")
    if len(parts) == 1:
        title = f"IP address locations targeting SSH in {parts[0]} networks"
    elif len(parts) == 2:
        title = f"IP address locations targeting SSH in {parts[0]} & {parts[1]} networks"
    else:
        title = f"IP address locations targeting SSH in {parts[0]}, {parts[1]} & {parts[2]} networks"
    
    v.plot(
        x='latitude',
        y='longitude',
        kind='scatter',
        c='requests',
        colormap="winter",
        ax=ax,
        title=title,
    )
    

In [ ]:
# 

import sqlite3
con = sqlite3.connect(":memory:")

for k in plots.keys():
    plots[k].to_sql(k.replace(" ", "_"), con, index=True)

    
#df_ip.to_sql("ip", con, index=False)
df_ip[['ip', 'city', 'country']].to_sql("ip", con, index=False)
#query = "SELECT ip, longitude, latitude FROM cloud INTERSECT SELECT ip, longitude, latitude FROM campus;"
#plots['cloud campus'] = pd.read_sql_query(query, con)

query = "SELECT * FROM campus"

countries = {}
cities = {}

for k in plots.keys():
#    query = f"""
#        SELECT x.ip, x.longitude, x.latitude, y.country, y.city, x.requests
#        FROM {k.replace(" ", "_")} as x, ip as y
#        WHERE x.ip = y.ip
#    """
    
    query = f"""
        SELECT x.ip, y.country, y.city, x.requests
        FROM {k.replace(" ", "_")} as x, ip as y
        WHERE x.ip = y.ip
    """

    tmp = pd.read_sql_query(query, con)
    
    cities[k] = pd.merge(
        tmp.groupby(['country', 'city']).count().sort_values(by='ip', ascending=False).drop('requests', axis=1),
        tmp.groupby(['country', 'city']).sum().sort_values(by='requests', ascending=False),
        how='left',
        on=['country', 'city'],
    ).sort_values(by='ip', ascending=False)
    
    countries[k] = pd.merge(
        tmp.drop('city', axis=1).groupby(['country']).count().sort_values(by='ip', ascending=False).drop('requests', axis=1),
        tmp.drop('city', axis=1).groupby(['country']).sum().sort_values(by='requests', ascending=False),
        how='inner',
        on='country',
    ).sort_values(by='ip', ascending=False)
    

for k in ips.keys():
    
    with pd.option_context(
        'display.max_rows', None,
        'display.max_columns', None
    ):
        print("="*32)
        print(f"No. of IPs targetting SSH in {', '.join(k.split(' '))} (by country)")
        print(countries[k].head(10))
        
        #print("\n")
        #print(f"No. of IPs targetting SSH in {', '.join(k.split(' '))} (by country and city)")
        #print(k)
        #print(cities[k].head(10))
        
        print("\n"*2)
    
